## Analyse der Employees-Datenbank

### Entity Relationship Diagram:

![Entity Relationship Diagram](./img/ERD_employees.png)

### Grundlegende Abfragen zu Mitarbeiterdaten

1. Wie viele salary-Werte hat die angestellte Fachkraft mit der Nummer emp_no '100001' erhalten?

In [ ]:
SELECT emp_no, salary
FROM salaries
WHERE emp_no = 100001;

2. Wie viele unterschiedliche Titel gibt es in dem Unternehmen?
- Experimentiere mit dem Befehl DISTINCT

In [ ]:
SELECT DISTINCT title FROM titles;

3. Wie viele Personen haben am 17. August Geburtstag? 
- Benutze einmal `birth_date::text LIKE ...` und einmal die Funktion `date_part`. 

In [ ]:
SELECT birth_date FROM employees
WHERE birth_date::text LIKE '%-08-17';
--
SELECT birth_date FROM employees
WHERE date_part('day', birth_date) = 17 AND date_part('month', birth_date) = 8;

4. Wie lauten die Nachnamen aller Angestellten, die eine emp_no kleiner als 20.000 haben, Uri mit Vornamen heißen, weiblich sind und, entweder zwischen 1957-05-23 und 1958-01-01 Geburtstag haben oder zwischen 1989-02-10 und 1990-01-01 eingestellt wurden.
- Benutze BETWEEN.

In [ ]:
SELECT last_name FROM employees
WHERE 
	emp_no < 20000 AND 
	first_name = 'Uri' AND 
	gender = 'F' AND 
	(birth_date BETWEEN '1957-05-23' and '1958-01-01' OR
	hire_date BETWEEN '1989-02-10' and '1990-01-01');

5. Gibt es weitere Mitarbeiter, die die gleichen Kriterien wie die in Aufgabe 4 erfüllen, jedoch nicht den Namen Uri tragen, sondern einen Vornamen mit drei Buchstaben haben, der mit U beginnt?

In [ ]:
SELECT * FROM employees
WHERE 
	emp_no < 20000 AND 
	first_name LIKE 'U__' AND 
	first_name != 'Uri' AND
	gender = 'F' AND 
	(birth_date BETWEEN '1957-05-23' and '1958-01-01' OR
	hire_date BETWEEN '1989-02-10' and '1990-01-01');

### Analyse von Gehältern und Abteilungen

1. Was ist das durchschnittliche Gehalt für jedes Geschlecht in den unterschiedlichen Abteilungen?

In [ ]:
SELECT 
	e.gender as Geschlecht,
	d.dept_name as Abteilung,
	avg(s.salary) as Durch_Gehalt

FROM employees e
INNER JOIN salaries s USING(emp_no)
INNER JOIN dept_emp de USING(emp_no)
INNER JOIN departments d USING(dept_no)

WHERE de.to_date > CURRENT_DATE AND s.to_date > CURRENT_DATE

GROUP BY Abteilung, Geschlecht
ORDER BY Abteilung, Geschlecht desc;

2. Welche Abteilung verdient im Durchschnitt am meisten (nach Geschlecht)?

In [ ]:
SELECT 
	e.gender as Geschlecht,
	d.dept_name as Abteilung,
	avg(s.salary) as Durch_Gehalt

FROM employees e
INNER JOIN salaries s USING(emp_no)
INNER JOIN dept_emp de USING(emp_no)
INNER JOIN departments d USING(dept_no)

WHERE de.to_date > CURRENT_DATE AND s.to_date > CURRENT_DATE
GROUP BY e.gender, d.dept_name
ORDER BY Durch_Gehalt desc;

3.  Wie viele Mitarbeiter arbeiten in den Abteilungen?

In [ ]:
SELECT 
	e.gender as Geschlecht,
	d.dept_name as Abteilung,
	avg(s.salary) as Durch_Gehalt,
	count(dept_no) as Anzahl_MAS

FROM employees e
INNER JOIN salaries s USING(emp_no)
INNER JOIN dept_emp de USING(emp_no)
INNER JOIN departments d USING(dept_no)

WHERE de.to_date > CURRENT_DATE AND s.to_date > CURRENT_DATE
GROUP BY e.gender, d.dept_name
ORDER BY Durch_Gehalt desc;

4. Welche und wie viele Abteilungs-Geschlecht-Kombinationen aus Aufgabe 1c haben mehr als 10.000 Mitarbeiter?

In [ ]:
SELECT 
	e.gender as Geschlecht,
	d.dept_name as Abteilung,
	avg(s.salary) as Durch_Gehalt,
	count(de.dept_no) as Anzahl_MAS

FROM employees e
INNER JOIN salaries s USING(emp_no)
INNER JOIN dept_emp de USING(emp_no)
INNER JOIN departments d USING(dept_no)

WHERE de.to_date > CURRENT_DATE AND s.to_date > CURRENT_DATE 
	
GROUP BY e.gender, d.dept_name
HAVING count(de.dept_no) > 10000
ORDER BY Durch_Gehalt desc;

### Fortgeschrittene Abfragen und Self-Joins

Erstelle eine SQL-Abfrage, die eine Tabelle erzeugt, in der Mitarbeitende als Paare gegenübergestellt werden. 
- Die Paare sollen das gleiche Geschlecht haben und im Jahr 1965 geboren sein. 
- Die Abfrage soll weder eine WHERE- noch eine HAVING-Bedingung verwenden. Stattdessen sollen alle notwendigen Bedingungen im ON-Clause der JOIN-Anweisung angegeben werden. Beachte dabei die reflexive Relation, d.h. die Tabelle soll keine Paare enthalten, bei denen Mitarbeitende mit sich selbst gepaart werden.

In [ ]:
SELECT  
  *
FROM employees e
INNER JOIN employees em ON e.emp_no < em.emp_no
			AND e.gender = em.gender
			AND EXTRACT(YEAR FROM e.birth_date) = 1965 
			AND EXTRACT(YEAR FROM em.birth_date) = 1965;